

```
Author ===> Jhanvi Shah
GitHub ===> @Jhanvi2001
Charotar University Of Science and Technology, CSPIT
Anand, Gujarat, India.
```



## **0. Libraries**

In [13]:
# Libraries/Modules
from tqdm import tqdm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from imutils import paths
import cv2
import os

In [12]:
# import the necessary packages
from skimage import feature
import numpy as np


class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns

        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1. Loading Data**

In [17]:
desc = LocalBinaryPatterns(24, 8)
data = []  # Will
labels = []
home_path = os.getcwd()
import glob
# loop over the training images
# imagePaths = list(paths.list_images('/content/drive/MyDrive/Dataset/LivDet 2015/Digital_Persona/Training/')) + list(paths.list_images('/content/drive/MyDrive/Dataset/LivDet 2015/Digital_Persona/Testing'))
imagePaths = glob.iglob('/content/drive/MyDrive/Dataset/LivDet 2015/Digital_Persona/' + '**/*.png', recursive=True)
# print(list(imagePaths))
# print(list(paths.list_images('testing')))
print("Done with storing the list of images")

print("Started image processing")
for imagePath in tqdm(imagePaths):
    # load the image, convert it to grayscale, and describe it
    gray = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
    hist = desc.describe(gray)

    # extract the label from the image path, then update the
    # label and data lists
    labels.append(imagePath.split(os.path.sep)[-2])
    data.append(hist)
print("Done with image processing")


Done with storing the list of images
Started image processing


308it [00:43,  7.08it/s]

Done with image processing


## **2. Support Vector Machine**

In [20]:

'''
Split the data into a training and testing set
Train: 80%
Test: 20%
'''
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True)

'''
Performing GridSearchCV with 5 folds to get the best value of parameters
'''
grid_search = GridSearchCV(SVC(kernel='rbf'),
                           param_grid={'C': [10, 100, 1000, 10000, 100000], 'gamma': [100, 10, 1, 0.1, 0.01, .0001]},
                           cv=5, verbose=True)
grid_search.fit(x_train, y_train)
params = grid_search.best_params_
print("Best parameter: ", params)

model = SVC(kernel='rbf', C=params['C'], gamma=params['gamma'], probability=True,verbose=True)
model.fit(x_train, y_train)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameter:  {'C': 100000, 'gamma': 0.1}
[LibSVM]

SVC(C=100000, gamma=0.1, probability=True, verbose=True)

In [26]:
#save the model with joblibpython 
import joblib
filename = 'SVM_rbf.h5'
joblib.dump(model, filename)

model.score(x_test, y_test)

0.8709677419354839

## **3. Performance Analysis**

In [27]:
print(model.predict(x_test[1].reshape(1, -1)))
print(model.predict_proba(x_test[1].reshape(1, -1)))
print("Test accuracy: ", model.score(x_test, y_test))
print("train accuracy: ", model.score(x_train, y_train))

['Live']
[[0.02408559 0.97068531 0.0052291 ]]
Test accuracy:  0.8709677419354839
train accuracy:  0.8902439024390244


## **4. Testing Phase**

In [46]:
desc = LocalBinaryPatterns(24, 8)
data = []  # Will
image_path = '/content/drive/MyDrive/Dataset/GitHub_Dataset/testing/Live/10_5.png'
gray = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
data = desc.describe(gray)
model = joblib.load('SVM_rbf.h5')
print(model.predict(data.reshape(1, -1)))

['Live']
